### Recomendação musical em decorrência do mapeamento facial

_Este projeto de pesquisa explora o contexto da Inteligência Artificial para a identificação de microexpressões faciais sutis, de menores dimensões, e involuntárias. Ainda, este projeto utiliza aspectos da musicoterapia, a fim de se viabilizar um sistema de recomendação musical._

### Bibliotecas necessárias
__________

In [5]:
import cv2
import numpy as np
import os
from deepface import DeepFace
import pandas as pd
from asyncio.windows_events import NULL
from genericpath import exists
from pandas import DataFrame
from sklearn.neighbors import NearestNeighbors
import math
import requests
import webbrowser
import spotipy


def GetRosto():
    backends = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface', 'mediapipe']
    vid = cv2.VideoCapture(0)
    while True:
        ret,img = vid.read()
        tela = np.array(img)
        frame = tela
        cv2.imshow('Capturando',frame)
        key=cv2.waitKey(1)
        if key == ord('q'):
            fname = "Temp\\frame.png" 
            cv2.imwrite(fname,frame)
            try:
                img = DeepFace.detectFace(fname,detector_backend=backends[3])
            except:
                continue
            
            os.remove(fname)
            img = img[:, :, ::-1]
            cv2.imshow('Capturando',img)
            min_val,max_val=img.min(),img.max()
            img = 255.0*(img - min_val)/(max_val - min_val)
            img = img.astype(np.uint8)
            cv2.imwrite(fname,img)
            while True:
                key=cv2.waitKey(1)
                if key == ord('q'):
                    break
            break
    cv2.destroyAllWindows()
    return fname

def get_face_param(img_param):
    df = pd.DataFrame(columns=list(img_param.keys()), data=[list(img_param.values())])
    for emotion in img_param['emotion']:
        df[emotion] = img_param['emotion'][emotion]
    df = df.drop(columns=['emotion','region', 'race'])
    return df

def get_face_image(image_path):
    fname = image_path

    try:
        predictions = DeepFace.analyze(fname, detector_backend='mtcnn')
    except:
        try:
            predictions = DeepFace.analyze(fname, detector_backend='opencv')
        except:
            print("Erro na leitura da imagem!")
            return 0
    return predictions

def training(database):

    df_train = database[['age', 'gender', 'angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']]
    df_train['gender'] = df_train['gender'].replace('Man',0).replace('Woman',1)

    X = df_train
    n_neighbors = 5
    knn = NearestNeighbors(n_neighbors=n_neighbors+1, metric= "cosine")
    knn.fit(X)
    return knn

def intervaloConfianca(df):

    df_int_conf = pd.DataFrame()

    for x in range(len(df.columns)):
        column=df[df.columns[x]]
        desvio=column.std(axis=None)
        err=desvio/math.sqrt(len(column))
        intervalo=err*1.96
        print(df.columns[x],"\nmin",column.mean()-intervalo,"max",column.mean()+intervalo)
        df_int_conf[df.columns[x]] = [column.mean()-intervalo, column.mean()+intervalo]
    return df_int_conf

def feedback(music_param, face_param):
    choice = input('Você gostou desta recomendação musical? [S/N]: ')
    choice = choice.lower()

    while(choice != 's' and choice != 'n'):
        print('Sua resposta tem que ser "S" ou "N" ')

        choice = input('Você gostou desta recomendação musical? [S/N]: ')
        choice = choice.lower()

    if(choice == 's'):
        # guarda dados
        df = face_param
        df = df.join(music_param)
        df.to_csv('database\databaseTest.csv', mode='a', header=False)
        return True
    else:
        #faz algo ainda
        return False

def listToString(list):
    finalString=""
    for x in range(len(list)):
        if(x == len(list)-1):
            finalString=finalString+list[x]
            return finalString
        finalString=list[x]+","+finalString
    return finalString


def play_music(musicId):
    
    f = open('Temp\index.html', 'w')
    
    html_template = """
    <html>
    <head>
    <title>Recomendação Musical</title>
    </head>
    <body>
    <iframe style="border-radius:12px" src="https://open.spotify.com/embed/track/{id}?utm_source=generator" width="100%" height="352" frameBorder="0" allowfullscreen="" allow="autoplay; clipboard-write; encrypted-media; fullscreen; picture-in-picture" loading="lazy"></iframe>
    
    </body>
    </html>
    """.format(id=musicId)
    
    f.write(html_template)
    
    f.close()
    
    filename = 'file:///'+os.getcwd()+'/' + 'Temp\index.html'
    webbrowser.open_new_tab(filename)

def post_music_request(intervalo_params, music_ids, artist_ids, artist_genres):

    int_min = intervalo_params.loc[0]
    int_max = intervalo_params.loc[1]

    url = 'http://localhost:5000/recommend/api/v1/get-recommendation'
    myobj = {
        "limit": "2",
        #"market":"ES",
        #"seed_artists": artist_ids,
        #"seed_genres": artist_genres,
        "seed_tracks": music_ids,
        #"min_danceability": str(int_min['danceability']),
        #"max_danceability": str(int_max['danceability']),
        #"min_energy": str(int_min['energy']),
        #"max_energy": str(int_max['energy']),
        #"min_acousticness": str(int_min['acousticness']),
        #"max_acousticness": str(int_max['acousticness']),
        #"min_instrumentalness": str(int_min['instrumentalness']),
        #"max_instrumentalness": str(int_max['instrumentalness']),
        #"min_loudness": str(int_min['loudness']),
        #"max_loudness": str(int_max['loudness']),
        #"min_speechiness": str(int_min['speechiness']),
        #"max_speechiness": str(int_max['speechiness']),
        #"min_liveness": str(int_min['liveness']),
        #"max_liveness": str(int_max['liveness']),
        #"max_valence": str(int_max['valence']),
        #"min_valence": str(int_min['valence']),
        #"max_tempo": str(int_max['tempo']),
        #"min_tempo": str(int_min['tempo'])
    }

    print(myobj)

    x = requests.post(url = url, json = myobj)
    return x

def main():

    database = pd.read_csv("database/database3.csv")
    print(database)

    #Treinamento
    knn = training(database)

    img1 = 'img_database_r\Homem\Pessoa 04\Bravo_r.png'

    #Imagem ou Câmera?
    pred = get_face_image(GetRosto())
    #pred = get_face_image(img1)

    if pred != 0:
        new_data = get_face_param(pred)

    rosto = new_data[['age', 'gender', 'angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']]
    rosto['gender'] = rosto['gender'].replace('Man',0).replace('Woman',1)
    print(rosto)

    item_selected = rosto.iloc[[0]]
    item_selected = item_selected[rosto.columns] 

    # Determine the neighbors
    d, neighbors = knn.kneighbors(item_selected.values.reshape(1, -1))
    neighbors = neighbors[0][1::] 
    d = d[0][1::] 

    print(neighbors)
    
    neighborsMusics = pd.DataFrame()

    for x in neighbors:
        print(database.iloc[[x]][['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']])
        neighborsMusics = neighborsMusics.append(database.iloc[[x]][['danceability', 'energy', 'loudness', 'speechiness','acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']])

    intervalo_params = intervaloConfianca(neighborsMusics)

    music_ids = []
    artist_ids = []
    artist_genres = []
    for y in neighbors:
        music_ids.append(database.iloc[y]['id'])
        artist_ids.append(database.iloc[y]['artist_id'])
        res = (database.iloc[y]['artist_genres']).strip('][').split(', ')
        for item in res:
            if(not(item.replace("'",'') in artist_genres)):
                artist_genres.append(item.replace("'",''))
        #artist_genres = [*artist_genres , *(database.iloc[y]['artist_genres'])]
        #print(artist_genres)
        
    str_music_ids = listToString(music_ids)
    str_artist_ids = listToString(artist_ids)
    str_artist_genres = listToString(artist_genres)

    #print(intervalo_params)
    #print(str_music_ids)
    #print(str_artist_ids)
    #print(str_artist_genres)

    #Spotify WebAPI
    musicaRecomendada = post_music_request(intervalo_params, str_music_ids, str_artist_ids, str_artist_genres)

    

In [6]:
database = pd.read_csv("database/database3.csv")
print(database)

#Treinamento
knn = training(database)

img1 = 'img_database_r\Homem\Pessoa 04\Bravo_r.png'

#Imagem ou Câmera?
pred = get_face_image(GetRosto())
#pred = get_face_image(img1)

if pred != 0:
    new_data = get_face_param(pred)

rosto = new_data[['age', 'gender', 'angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']]
rosto['gender'] = rosto['gender'].replace('Man',0).replace('Woman',1)
print(rosto)

item_selected = rosto.iloc[[0]]
item_selected = item_selected[rosto.columns] 

# Determine the neighbors
d, neighbors = knn.kneighbors(item_selected.values.reshape(1, -1))
neighbors = neighbors[0][1::] 
d = d[0][1::] 

print(neighbors)

neighborsMusics = pd.DataFrame()

for x in neighbors:
    print(database.iloc[[x]][['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']])
    neighborsMusics = neighborsMusics.append(database.iloc[[x]][['danceability', 'energy', 'loudness', 'speechiness','acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']])

intervalo_params = intervaloConfianca(neighborsMusics)

music_ids = []
artist_ids = []
artist_genres = []
for y in neighbors:
    music_ids.append(database.iloc[y]['id'])
    artist_ids.append(database.iloc[y]['artist_id'])
    res = (database.iloc[y]['artist_genres']).strip('][').split(', ')
    for item in res:
        if(not(item.replace("'",'') in artist_genres)):
            artist_genres.append(item.replace("'",''))
    #artist_genres = [*artist_genres , *(database.iloc[y]['artist_genres'])]
    #print(artist_genres)
    
str_music_ids = listToString(music_ids)
str_artist_ids = listToString(artist_ids)
str_artist_genres = listToString(artist_genres)

#print(intervalo_params)
#print(str_music_ids)
#print(str_artist_ids)
#print(str_artist_genres)

#Spotify WebAPI
musicaRecomendada = post_music_request(intervalo_params, str_music_ids, str_artist_ids, str_artist_genres)

    Unnamed: 0  age gender      angry       disgust      fear      happy  \
0            0   32    Man  92.755395  1.742503e-03  1.530258   0.000033   
1            1   27    Man   0.025171  1.466431e-04  0.010573  96.789300   
2            2   29    Man  45.823449  4.112436e-03  2.985845   0.000594   
3            3   30    Man  66.152805  6.050717e-02  8.196913   0.107843   
4            4   34    Man   1.954687  1.198679e-03  1.221227   0.013479   
..         ...  ...    ...        ...           ...       ...        ...   
78          78   31    Man  53.223222  3.628170e-04  0.606637   0.340197   
79          79   32    Man   0.432744  1.064093e-02  0.016589   0.024746   
80          80   32    Man   0.000514  5.052229e-08  0.000926  79.427618   
81          81   32    Man   0.432744  1.064093e-02  0.016589   0.024746   
82          82   31    Man   0.032347  2.066264e-05  0.012612   0.029215   

          sad   surprise    neutral  ...                      id  \
0    5.704149   0.0

Action: race: 100%|██████████| 4/4 [00:01<00:00,  3.20it/s]   


   age  gender     angry   disgust      fear     happy       sad  surprise  \
0   31       0  0.073302  0.000001  1.931991  0.286854  4.748942  0.009283   

     neutral  
0  92.949629  
[81 41 10 71 18]
    danceability  energy  loudness  speechiness  acousticness  \
81         0.508   0.471   -10.718       0.0385         0.783   

    instrumentalness  liveness  valence   tempo  
81                 0      0.25    0.117  81.995  
    danceability  energy  loudness  speechiness  acousticness  \
41         0.484   0.039   -28.557        0.048         0.991   

    instrumentalness  liveness  valence    tempo  
41                 0     0.133    0.118  112.814  
    danceability  energy  loudness  speechiness  acousticness  \
10         0.784  0.0752   -13.934        0.142         0.928   

    instrumentalness  liveness  valence  tempo  
10                 0     0.105    0.476  81.97  
    danceability  energy  loudness  speechiness  acousticness  \
71         0.743   0.836    -6.465    

In [18]:
import json
musicaRecomendada.json()[0]["id"]

'1QKUHzGaOAxqx2z2AxlVDO'

In [2]:
main()

    Unnamed: 0  age gender      angry       disgust      fear      happy  \
0            0   32    Man  92.755395  1.742503e-03  1.530258   0.000033   
1            1   27    Man   0.025171  1.466431e-04  0.010573  96.789300   
2            2   29    Man  45.823449  4.112436e-03  2.985845   0.000594   
3            3   30    Man  66.152805  6.050717e-02  8.196913   0.107843   
4            4   34    Man   1.954687  1.198679e-03  1.221227   0.013479   
..         ...  ...    ...        ...           ...       ...        ...   
78          78   31    Man  53.223222  3.628170e-04  0.606637   0.340197   
79          79   32    Man   0.432744  1.064093e-02  0.016589   0.024746   
80          80   32    Man   0.000514  5.052229e-08  0.000926  79.427618   
81          81   32    Man   0.432744  1.064093e-02  0.016589   0.024746   
82          82   31    Man   0.032347  2.066264e-05  0.012612   0.029215   

          sad   surprise    neutral  ...                      id  \
0    5.704149   0.0

Action: race: 100%|██████████| 4/4 [00:01<00:00,  2.75it/s]   


   age  gender     angry   disgust       fear     happy        sad  surprise  \
0   29       0  1.475399  0.000103  15.839786  0.046541  77.908089  0.004378   

    neutral  
0  4.725702  
[48 54 50 46 58]
    danceability  energy  loudness  speechiness  acousticness  \
48         0.782   0.828     -4.22       0.0428        0.0361   

    instrumentalness  liveness  valence    tempo  
48                 0     0.142    0.839  141.872  
    danceability  energy  loudness  speechiness  acousticness  \
54         0.687   0.792    -2.749       0.0452         0.191   

    instrumentalness  liveness  valence    tempo  
54                 0     0.167    0.671  100.015  
    danceability  energy  loudness  speechiness  acousticness  \
50         0.639   0.526    -6.697       0.0256         0.245   

    instrumentalness  liveness  valence    tempo  
50                 0      0.25    0.488  130.033  
    danceability  energy  loudness  speechiness  acousticness  \
46         0.601   0.741    -5

In [1]:
import cv2 #pip install opencv-python 

from deepface import DeepFace #pip install deepface

### Input (vídeo/foto)
_______

### Dedução da emoção predominante
______

### Recomendação musical 

________

### Resposta do usuário

_______

### Cálculo do erro
________